<a href="https://colab.research.google.com/github/sean-otoole/HODD/blob/main/geneformer_training_with_dop_neuron_pop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install scanpy anndata umap-learn python-igraph louvain h5py leidenalg > /dev/null 2>&1

In [2]:
from google.colab import drive
import os
import scanpy as sc
import pandas as pd
from scipy.io import mmread
import scipy.sparse
import pickle

In [23]:
# drive.mount('/content/drive')
# os.chdir("/content/drive/MyDrive/HODD/")

drive.flush_and_unmount()  # Unmount Google Drive
drive.mount('/content/drive')  # Remount it
os.chdir("/content/drive/MyDrive/HODD/")


Mounted at /content/drive


In [4]:
!git lfs install

Git LFS initialized.


In [5]:
!git clone https://huggingface.co/ctheodoris/Geneformer

fatal: destination path 'Geneformer' already exists and is not an empty directory.


In [6]:
%cd Geneformer

/content/drive/MyDrive/HODD/Geneformer


In [7]:
!pip install .

Processing /content/drive/MyDrive/HODD/Geneformer
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664

In [25]:
from geneformer import TranscriptomeTokenizer

# Prepare the data using Geneformer Preprocessor

data_directory = '/content/drive/MyDrive/HODD/Geneformer_parkinsons_files'
output_path = data_directory
ouput_prefix = 'dopamine_subset_tokenized'

tk = TranscriptomeTokenizer({"disease_state": "disease_state"}, nproc=4)
tk.tokenize_data(data_directory, output_path, ouput_prefix, file_format="h5ad")

Tokenizing /content/drive/MyDrive/HODD/Geneformer_parkinsons_files/dopamine_subset_pre_tokenization.h5ad
/content/drive/MyDrive/HODD/Geneformer_parkinsons_files/dopamine_subset_pre_tokenization.h5ad has no column attribute 'filter_pass'; tokenizing all cells.


/content/drive/MyDrive/HODD/Geneformer/geneformer/tokenizer.py:495: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  for i in adata.var["ensembl_id_collapsed"][coding_miRNA_loc]
/content/drive/MyDrive/HODD/Geneformer/geneformer/tokenizer.py:498: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  coding_miRNA_ids = adata.var["ensembl_id_collapsed"][coding_miRNA_loc]


TypeError: Object with dtype category cannot perform the numpy op divide

In [ ]:
from geneformer import TranscriptomeTokenizer

In [ ]:
from geneformer import Geneformer

# Load pretrained Geneformer model
model = Geneformer.from_pretrained("geneformer")

# Fine-tune on your dataset
model.train(tokenized_data, output_dir="fine_tuned_geneformer")

In [ ]:
perturbed_data = preprocessor.apply_perturbation(adata.X, genes_to_perturb=["GeneA", "GeneB"])
perturbed_predictions = model.predict(perturbed_data)


In [18]:
import scanpy as sc

file_path = "/content/drive/MyDrive/HODD/Geneformer_parkinsons_files/dopamine_subset_pre_tokenization.h5ad"
data = sc.read_h5ad(file_path)

# Check the columns in `data.var`
print(data.var.columns)

# Check if `ensembl_id` is actually present
if "ensembl_id" in data.var.columns:
    print("ensembl_id column is present.")
else:
    print("ensembl_id column is missing.")


FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = '/content/drive/MyDrive/HODD/Geneformer_parkinsons_files/dopamine_subset_pre_tokenization.h5ad', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)